### Authentication and other code admin

* Importing Python libraries
* Setting up Google Cloud Platform (GCP) services
    * Service account impersonation authentication and secrets retrieval 
    * Cloud storage access and file retrieval
* Getting files organized

#### Importing Python libraries

In [5]:
#data org/manipulation
import pandas as tian
tian.set_option('display.max_columns', None)
from io import StringIO
import builtins
from tqdm import tqdm
import pickle
import os


#gcp
from google.auth import default, impersonated_credentials
from google.cloud import secretmanager, storage

#llm libraries
from transformers import AutoTokenizer
from langchain import document_loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import random


#### Setting up Google Cloud Platform (GCP) services

##### Service account impersonation authentication and secrets retrieval 

You will need to change this to your preferred setup for accessing files and secrets if you want to follow along. My first start is authenticating into GCP from the command line:

```
gcloud auth application-default login
```


In [3]:
# Default credentials from the environment
credentials, project_id = default()
# service account impersonation setup
service_account_email = "gcp-music-service@thoughts-on-music.iam.gserviceaccount.com"
target_scopes = ['https://www.googleapis.com/auth/cloud-platform']

impersonated_credentials = impersonated_credentials.Credentials(
    source_credentials=credentials,
    target_principal=service_account_email,
    target_scopes=target_scopes
)

# secrets client
secret_client = secretmanager.SecretManagerServiceClient()
project_id = "thoughts-on-music"
secret_id = "HUGGING_FACE_READ_TOKEN_LLAMA"
version_id = "latest" 
secrets_name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

# retrieve HF API key from secrets manager
response = secret_client.access_secret_version(name=secrets_name)
HUGGING_FACE_READ_TOKEN_LLAMA = response.payload.data.decode("UTF-8")


##### Cloud storage access and file admin

I've defined a couple functions for used as needed:

* gcp_download takes a defined GPC bucket name, folder location, and file name and returns a dataframe of that file
    * It's designed so other bucket and folder names can be introduced without wrecking the code
* df_deets takes either a single dataframe name or a list of dataframe name and prints out details I like:
    * The name of the dataframe
    * The first few rows
    * The column names
    * The data types for each field
    * Sum of duplicates and NAs for each column 

In [ ]:
# cloud storage client setup
storage_client = storage.Client(credentials=credentials)
love_uwsthoughts = "love-uwsthoughts"
bp_csvs_folder = 'bp_csvs'
bp_artist_bios_folder = 'bp_artist_bios'
uwsthoughts_bucket = storage_client.bucket(love_uwsthoughts)

#function to import files 
def gcp_download(bucket_name, gcp_folder, file_name):
    bucket = storage_client.bucket(bucket_name)
    blob_name = f"{gcp_folder}/{file_name}".strip("/")
    file_blob = bucket.blob(blob_name)
    file_data = file_blob.download_as_text() 
    gcp_df = tian.read_csv(StringIO(file_data))
    
    return gcp_df

def df_deets(df_or_names):
    def print_info(df, df_name):
        print(f"{df_name}:")
        if hasattr(builtins, "display"):
            display(df)
        else:
            print(df)
        print("\n")
        
        # Summary of data types, duplicates, NAs, and distinct values
        summary_df = tian.DataFrame({
            "Data types": df.dtypes,
            "Duplicates": [df.duplicated(subset=[col]).sum() for col in df.columns],
            "NAs": df.isna().sum().values,
            "Distinct values": df.nunique().values
        }).set_index(df.columns)
        
        print(f"{df_name} summary:")
        if hasattr(builtins, "display"):
            display(summary_df)
        else:
            print(summary_df)
        print("\n")
        print(df.columns)
        print("\n")
        total_duplicates = df.duplicated().sum()
        print(f"Total number of duplicate rows in {df_name}: {total_duplicates}\n")

    if isinstance(df_or_names, list): 
        for item in df_or_names:
            if isinstance(item, str):
                df = globals()[item] 
                print_info(df, item)
            else:
                print_info(item, "dataframe")
    else: 
        if isinstance(df_or_names, str):
            df = globals()[df_or_names]  
            print_info(df, df_or_names)
        else:
            print_info(df_or_names, "dataframe")


##### Getting files organized 

This may seem beautifully consolidated now but trust me when I say that there was a lot of chaos I went through to get to this point. I keep like with like to make it easier for me to remember what I was thinking.

Running list of different files I need at various points:

In [ ]:
fact_bp_track_audio_features_csv = 'fact_bp_track_audio_features.csv' 
fact_bpmeta_audio_csv = 'fact_bpmeta_audio.csv'
fact_label_bpmeta_audio_csv = 'fact_label_bpmeta_audio.csv'
bp_artist_release_csv = 'bp_artist_release.csv'
bp_artist_csv = 'bp_artist.csv'
bp_artist_track_csv = 'bp_artist_track.csv'
bp_label_csv = 'bp_label.csv'
bp_genre_csv = 'bp_genre.csv'
bp_key_csv = 'bp_key.csv'
bp_label_artist_csv = 'bp_label_artist.csv'

I only had to download from GCP once and then I cached the ones I need for later use. If I make a change that I want to be permanent, I update the cache with the changes. From time to time I'll send a changed file up to GCP if I have a long term need for it. 

In [ ]:
bp_artist_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_artist_csv)
bp_artist_release_df =  gcp_download(love_uwsthoughts, bp_csvs_folder, bp_artist_release_csv)
bp_artist_track_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_artist_track_csv)

bp_label_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_label_csv)
bp_label_artist_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_label_artist_csv)

bp_genre_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_genre_csv)
bp_key_df = gcp_download(love_uwsthoughts, bp_csvs_folder, bp_key_csv)
fact_label_bpmeta_audio_df = gcp_download(love_uwsthoughts, bp_csvs_folder, fact_label_bpmeta_audio_csv)

fact_bp_track_audio_features_df = gcp_download(love_uwsthoughts, bp_csvs_folder, fact_bp_track_audio_features_csv)
fact_bpmeta_audio_df = gcp_download(love_uwsthoughts, bp_csvs_folder, fact_bpmeta_audio_csv)

bp_artist_df.to_parquet('bp_artist_df_cache.parquet', index=False)
bp_artist_release_df.to_parquet('bp_artist_release_df_cache.parquet', index=False)
bp_artist_track_df.to_parquet('bp_artist_track_df_cache.parquet', index=False)

bp_label_df.to_parquet('bp_label_df_cache.parquet', index=False)
bp_label_artist_df.to_parquet('bp_label_artist_df_cache.parquet', index=False)

bp_genre_df.to_parquet('bp_genre_df_cache.parquet', index=False)
bp_key_df.to_parquet('bp_key_df_cache.parquet', index=False)



A consolidated list of the cached dataframes I need:

In [2]:
# bpmeta_audio_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bpmeta_audio_df_cache.parquet') # removed fact from name
# bpmeta_audio_shield_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bpmeta_audio_shield_df_cache.parquet')

# bp_artist_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_artist_df_cache.parquet')
# bp_artist_release_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_artist_release_df_cache.parquet')
# bp_artist_track_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_artist_track_df_cache.parquet')

# bp_artist_label_names_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_artist_label_names_df_cache.parquet')
# artist_bios_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/artist_bios_df_cache.parquet')

# bp_label_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_label_df_cache.parquet')
# bp_label_artist_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_label_artist_df_cache.parquet')

# bp_genre_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_genre_df_cache.parquet')
# bp_key_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_key_df_cache.parquet')

bp_text_values_df = tian.read_parquet('/Users/uwsthoughts/Desktop/dolly_shield_local/bp_text_values_df_cache.parquet')



### Data here, there, and everywhere

This first section is about futzing around with files I made when I was much younger, maybe early summer 2024? I'm going to sweep through and see which ones are actually valuable and which ones are the result of me wrapping myself in a massive vortext of delusion.

In [ ]:
fact_bp_track_audio_features_df = gcp_download(love_uwsthoughts, bp_csvs_folder, fact_bp_track_audio_features_csv)
fact_bpmeta_audio_df = gcp_download(love_uwsthoughts, bp_csvs_folder, fact_bpmeta_audio_csv)

In [ ]:
dataframes_list = ['fact_bp_track_audio_features_df', 'fact_bpmeta_audio_df']

df_deets(dataframes_list)

Based on the above, it looks like I intended for 'fact_bpmeta_audio_df' to be the cleaned up version of 'fact_bp_track_audio_features_df'. There were ~13,000 rows without Beatport track_id's and, given that this heavily revolves around Beatport data, I decided that it wasn't worth keeping records without that ID. While I do want to get the show on the road, I do have to return to  'mix' having 477,121 distinct values. The table above already has all the cleaned up values in place but below is how I previously did it. I've also added some data to show that we are dealing with the longest of longest tails for this one.

In [ ]:
bpmeta_audio_df = tian.merge(
    fact_bpmeta_audio_df,
    fact_bp_track_audio_features_df[['track_id', 'title', 'track_url']],
    on='track_id',
    how='left'
)

bpmeta_audio_df

In [ ]:
mix_replacements = {
    "continuous": "Set Mixed",
    "live": "Set Mixed",
    "remastered": "Remastered Mix",
    "orginal": "Original Mix",
    "ambient": "Ambient Mix",
    "chill": "Ambient Mix",
    "lounge": "Ambient Mix",
    "rework": "Remastered Mix",
    "remix": "Remix",
    "original": "Original Mix",
    "club": "Club Mix",
    "dub": "Dub Mix",
    "extended": "Extended Mix",
    "instrumental": "Instrumental Mix",
    "radio": "Radio Mix",
    "vip": "Remix",
    "album": "Album Mix",
    "Continuous DJ Mix": "Mixed",
    "Mix Cut": "Set Mixed",
    "Mixed": "Set Mixed",
    "Intro Mix": "Set Mixed",
    "Edit": "Radio Edit",
    "Main Mix": "Original Mix",
    "Album Version": "Album Mix",
    "Deep Mix": "Remix",
    "House Mix": "Remix",
    "Tribal Mix": "Remix",
    "Intro": "Set Mixed",
    "Edit Mix": "Radio Mix",
    "Bonus Track": "Album Mix"
}

# Function to clean up mix values
def clean_mix_values(mix):
    if isinstance(mix, str):
        for key, value in mix_replacements.items():
            if key.lower() in mix.lower():
                return value
    return mix


bpmeta_audio_df['mix'] = bpmeta_audio_df['mix'].apply(clean_mix_values)
total_rows = len(bpmeta_audio_df)
top_20 = bpmeta_audio_df['mix'].value_counts().head(20)
top_20_df = top_20.reset_index(name='count').rename(columns={'index': 'mix'})
top_20_df['percent_of_total'] = (top_20_df['count'] / total_rows) * 100

print("\nTop 20 mix values after cleanup:")
print(top_20_df)


I made the cache for bpmeta_audio_df down here and then moved the read cache part near the top with the rest. 

In [ ]:
bpmeta_audio_df.to_parquet('bpmeta_audio_df_cache.parquet', index=False)

### A suspicious label data table

I had an inkling that this old table I made was a botched attempt at joining label data to the other metadata. 

In [ ]:
df_deets(fact_label_bpmeta_audio_df)

Based on the above, that's exactly what happened. I'm willing to bet I can just append label_name onto fact_bpmeta_audio_df and have that be that.

In [ ]:
bpmeta_audio_shield_df = tian.merge(bpmeta_audio_df, fact_label_bpmeta_audio_df[['track_id', 'label_name']],
                     on='track_id', how='left')

bpmeta_audio_shield_df 

In [ ]:
bpmeta_audio_shield_df.to_parquet('bpmeta_audio_shield_df_cache.parquet', index=False)

df_deets(bpmeta_audio_shield_df)

### Artist bios integration

I scraped some aritst bios from Beatport in batches of 1,000 that were initially stored in separate files. This is where I bring them all together and have one list.

In [ ]:
#sort folder by created date and then grab
def gcp_folder_sort(bucket, folder):
    blobs = list(bucket.list_blobs(prefix=folder))
    sorted_blobs = sorted(blobs, key=lambda x: x.time_created)
    return sorted_blobs

#combine csv files into one df
def artists_united(bucket, folder):
    sorted_blobs = gcp_folder_sort(bucket, folder)
    artists_united_df = tian.DataFrame()
    for blob in sorted_blobs:
        if blob.name.endswith('.csv'):
            the_drop = gcp_download(bucket.name, folder, blob.name.split('/')[-1])
            the_drop = the_drop[['beatport_artist_id', 'artist_name', 'beatport_bio']]
            artists_united_df = tian.concat([artists_united_df, the_drop], ignore_index=True)
    artists_united_df = artists_united_df.drop_duplicates(ignore_index=True)        
    return artists_united_df

#save df back to gcp
def paranoid_guard(df, bucket_name, gcp_folder, file_name):
    bucket = storage_client.bucket(bucket_name)
    csv_data = df.to_csv(index=False)
    blob_name = f"{gcp_folder}/{file_name}".strip("/")
    blob = bucket.blob(blob_name)
    blob.upload_from_string(csv_data, content_type='text/csv')


In [ ]:
bppoints_artist_bios_csv = "bppoints_artist_bios.csv"

bppoints_artist_bios_df = artists_united(uwsthoughts_bucket, bp_artist_bios_folder)

paranoid_guard(bppoints_artist_bios_df, love_uwsthoughts, bp_artist_bios_folder, bppoints_artist_bios_csv)

I decided I was only interested in the artists that had a bio so of the ~55,000 I started with, I ended up with ~48,000. 

In [ ]:
artist_bios_df = bppoints_artist_bios_df.drop_duplicates(subset='beatport_bio', keep='first')

df_deets(artist_bios_df)

In [ ]:
artist_bios_df.to_csv('/Users/uwsthoughts/Desktop/bp_spotify_raw_data/csv_data/artist_bios_df.csv', index=False)
artist_bios_df.to_parquet('artist_bios_df_cache.parquet', index=False)

### A big ol' effort to make human readable text

A lot of my previous work has been premised on using different IDs to establish realtionships between data, with text values only coming in at the very end to help identify outputs. Now that I'm shifting into trying to use a large language model, I want text values instead of IDs so the model can tokenize and understand what it means. My thinking is that these IDs probably won't mean too much to it but it's probably been trained on enough data that it can make relationships with the addition data I give it. 

Below are a a few pieces that take me from ID mappings to human readable. I've never worked directly with an LLM before but my thinking is this: the human brain is essentially machine learning, perfected. In most languages, text is read left to right, top to bottom. If I create tables where the data can be read and understood by a human in a left to right fasion, an LLM should be able to do the same.

That means I need to take a bunch of data I have in different palces and bring it all together into a few, human readable tables.

#### Data sample reference

This is just a holding pen for samples of all the data I'm working with in this section. I used it as a reference point to confirm I was taking the right values from the right table. You can skip to 'The data in play' to see the actual work.

In [ ]:
df_list = [bpmeta_audio_shield_df, bp_artist_df, bp_artist_release_df, bp_artist_track_df, bp_label_df, bp_label_artist_df, bp_genre_df, bp_key_df]
df_deets(df_list)

#### Miami = synergy

While each dataframe has it's column names printed as part of df_deets() above, for this specific part I needed all the column names neatly stacked on top of each other so I could figure out what needed to go where. I've been working with this data for a few months now so all these tables and fields mean something to me, with the samples above used as needed. The only one that isn't an original dataset is```bpmeta_audio_shield_df```.

In [ ]:
df_names = ["bpmeta_audio_shield_df", "bp_artist_df", "bp_artist_release_df", "bp_artist_track_df", "bp_label_df", "bp_label_artist_df", "bp_genre_df", "bp_key_df"]

for name, df in zip(df_names, df_list):
    print(f"{name}:\n {df.columns}\n")

After looking at the above, I decided the best approach was to use ```bpmeta_audio_shield_df``` as the starting point and add in text values where it made sense. I did this with a couple of goals in mind:
* Make copies of tables I want to change so that I can preserve original tables in their original structure
* Combine data that can realistically go together, like the data saying which artists are on which labels

#### From the underground to the world 

This first part is straight forward: there were separate tables for artist and label metadata and I wanted to see a label and its artists together. I thought of it has "Anjunadeep has Marsh and Eli & Fur as artists." I kept the IDs and urls but tucked them in at the end. My spider sense says having those urls available later is going to be useful.

In [ ]:
label_artist_temp = tian.merge(bp_label_artist_df, bp_label_df, on='label_id', suffixes=('', '_label'))
bp_artist_label_names_df = tian.merge(label_artist_temp, bp_artist_df, on='artist_id', suffixes=('', '_artist'))
bp_artist_label_names_df = bp_artist_label_names_df[[ 'label_name', 'artist_name', 'label_id', 'label_url', 'artist_id', 'artist_url']]


In [ ]:
df_deets(bp_artist_label_names_df)

In [ ]:
bp_artist_label_names_df.to_parquet('bp_artist_label_names_df_cache.parquet', index=False)


### Four to the floor

Take a deep breath with me. From the above, I wanted to be able to say something like the following:

> On August 18th, 2023, Mira released 'Celo' on Kiosk ID. Her first release in 2023, it was also her first foray into melodic house & techno. It brought a newer and edgier side to her growing repertoire.

That led me to a table design that look like this:

- release_date
- artist_name
- title
- label_name
- genre_name
- bpm
- key_id
- mix
- is_remixed
- is_remixer
- mode
- valence
- danceability
- energy
- speechiness
- loudness
- liveness
- instrumentalness
- acousticness
- isrc
- artist_id
- artist_url
- track_id
- track_url
- label_id
- label_url
- genre_id
- genre_url

All of the text values are one after another on the left, with IDs and urls at the end. I have text value descriptions for all these Spotift audio metrics so it's easy to see how those go from continuous 0-1 values to something a human can understand. But,as they say, that's a problem for a different day.

To do this, I simply joined on the different IDs I needed, with track_id being the center of gravity. For now, I decided to have one track ID for each artist, which means multiple track IDs for each artist. In future work, I want to put all the artists together since the human brain doesn't separate out artists on a track like that. 


In [ ]:
bp_track_artist_merge = tian.merge(bpmeta_audio_shield_df, bp_artist_track_df, on='track_id', suffixes=('', '_artist'))
bp_track_artist_label_merge = tian.merge(bp_track_artist_merge, bp_artist_df, on='artist_id', suffixes=('', '_artist_info'))
bp_track_artist_label_merge = tian.merge(bp_track_artist_label_merge, bp_label_df, on='label_id', suffixes=('', '_label'))
bp_track_artist_label_merge = tian.merge(bp_track_artist_label_merge, bp_genre_df, on='genre_id', suffixes=('', '_genre'))
bp_text_values_df = bp_track_artist_label_merge[[
    'release_date', 'artist_name', 'title',  'label_name', 'duration', 'genre_name', 'bpm', 'key_id', 'mix', 'is_remixed', 'is_remixer',  
    'mode', 'valence', 'danceability', 'energy', 'speechiness', 'loudness', 'liveness', 'instrumentalness', 'acousticness', 'isrc', 'artist_id', 'artist_url', 
    'track_id', 'track_url', 'label_id', 'label_url', 'genre_id', 'genre_url'
]]

This brought me this beatifully massive table. On spec, it looks how I wanted. I need to handle things like ```is_remixed``` and ```is_remixer``` since they're telling the same story from different perspectives.

In [ ]:
df_deets(bp_text_values_df)

All of that work means I can also find artists I'm looking for much easier. Text values as IDs are, generally speaking, not great because it's easy to have overlaps and duplicates. For example, ```Mira (Berlin)``` has (Berlin) after her name because 1) she's from Berlin so it clearly identifies her and there's another ```Mira``` on Beatport and so someone has to change a bit. This is the divergence between the relational database world I've lived in and the more unstructured world of LLMs. I also didn't deal with ```bp_key_df``` meaningfully because it's a whole separate chunk of work to make it ready to add to this one. The key_id was preserved so I can easily return to it when ready. 

In [ ]:
# bp_text_values_df[(bp_text_values_df['artist_name'] == 'Fideles') & (bp_text_values_df['title'] == 'Away With Me')].sort_values(by='release_date', ascending=False)
bp_text_values_df[bp_text_values_df['artist_name'] == 'Mira (Berlin)'].sort_values(by='release_date', ascending=False).head(10)


In [ ]:
bp_text_values_df.to_parquet('bp_text_values_df_cache.parquet', index=False)


## Tokenizing Work

Initial work using Hugging Face and Langchain to access and use the folllowing open source Large Language Models (LLMs):

* [Meta's Llama 3.1-8B](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1#llama-3.1)
* [Meta's Llama-3.2-11B-Vision](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2#-llama-3.2-vision-models-(11b/90b)-)

First, I'm gonna bring in the files I cached. If you're following along at home, this notebook is coming together over several different sessions and I need a mechanism to bring the data back. I could redownload from GCP but that seems excessive.

### If we say tokens enough times, they appear

In [6]:
llama_318b = "meta-llama/Meta-Llama-3.1-8B"
llama_318b_tokenizer = AutoTokenizer.from_pretrained(llama_318b, token="HUGGING_FACE_READ_TOKEN_LLAMA")

llama_3211b = "meta-llama/Llama-3.2-11B-Vision"
llama_3211b_tokenizer = AutoTokenizer.from_pretrained(llama_3211b)

logging.basicConfig(filename='/Users/uwsthoughts/Desktop/dolly_shield_local/tokenization_logs.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def melodies(row):
    text = (
        f"Track ID: {row['track_id']}, Title: {row['title']}, "
        f"Artist: {row['artist_name']}, Artist ID: {row['artist_id']}, "
        f"Genre: {row['genre_name']}, Genre ID: {row['genre_id']}, "
        f"Label: {row['label_name']}, Label ID: {row['label_id']}, "
        f"Release Date: {row['release_date']}, Track URL: {row['track_url']}, "
        f"Mix: {row['mix']}, Remix: {'Yes' if row['is_remixed'] else 'No'}, "
        f"Remixer: {'Yes' if row['is_remixer'] else 'No'}, Duration: {row['duration']} minutes, "
        f"BPM: {row['bpm']}, Key ID: {row['key_id']}, "
        f"Mode: {row['mode']}, Valence: {row['valence']}, Danceability: {row['danceability']}, "
        f"Energy: {row['energy']}, Speechiness: {row['speechiness']}, "
        f"Loudness: {row['loudness']}, Liveness: {row['liveness']}, "
        f"Instrumentalness: {row['instrumentalness']}, Acousticness: {row['acousticness']}, "
        f"ISRC: {row['isrc']}, Artist URL: {row['artist_url']}, Label URL: {row['label_url']}, "
        f"Genre URL: {row['genre_url']}."
    )
    return text

def find_your_flow(dataframe, farm_trips):
    hay_barrels = (len(dataframe) // farm_trips) + 1
    full_shed = []

    for i in range(0, hay_barrels, 20):
        last_barrel = min(i + 20, hay_barrels)
        for barrel in tqdm(range(i, last_barrel), desc=f"Processing batches {i+1}-{last_barrel}"):
            fresh_cut = barrel * farm_trips
            full_up = min((barrel + 1) * farm_trips, len(dataframe))
            batch_df = dataframe.iloc[fresh_cut:full_up]
            
            full_barrel = [melodies(row) for _, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f"Processing batch {barrel+1}/{hay_barrels}")]
            full_shed.extend(full_barrel)
    
    return full_shed


def dolly_tunes(texts, farm_trips):
    hay_barrels = (len(texts) // farm_trips) + 1
    dolly_grazing = []

    #sample of 100 random rows 
    sample_indices = random.sample(range(len(texts)), min(100, len(texts)))

    for i in range(0, hay_barrels, 20):
        last_barrel = min(i + 20, hay_barrels)
        for barrel in tqdm(range(i, last_barrel), desc=f"Tokenizing batches {i+1}-{last_barrel}"):
            fresh_cut = barrel * farm_trips
            full_up = min((barrel + 1) * farm_trips, len(texts))
            full_barrel = texts[fresh_cut:full_up]
            
            #Tokenization of batch
            full_truck = llama_3211b_tokenizer(
                full_barrel,
                return_tensors="tf",
                truncation=True,
                padding=True
            )

            for idx, text in enumerate(full_barrel):
                tokens = llama_3211b_tokenizer.tokenize(text)
                token_ids = llama_3211b_tokenizer.convert_tokens_to_ids(tokens)

                #log all tokens into file
                logging.info(f"Text: {text}\nTokens: {tokens}\nToken IDs: {token_ids}")

                #print out match rows of random sample using global index
                global_idx = fresh_cut + idx  
                if global_idx in sample_indices:
                    print("Tokens:", tokens)
                    print("Token IDs:", token_ids)

            dolly_grazing.append(full_truck)

    return dolly_grazing

# def dolly_tunes(texts, farm_trips):
#     hay_barrels = (len(texts) // farm_trips) + 1
#     dolly_grazing = []

#     for i in range(0, hay_barrels, 20):
#         last_barrel = min(i + 20, hay_barrels)
#         for barrel in tqdm(range(i, last_barrel), desc=f"Tokenizing batches {i+1}-{last_barrel}"):
#             fresh_cut = barrel * farm_trips
#             full_up = min((barrel + 1) * farm_trips, len(texts))
#             full_barrel = texts[fresh_cut:full_up]
            
#             full_truck = llama_3211b_tokenizer(full_barrel, return_tensors="tf", truncation=True, padding=True)
#             dolly_grazing.append(full_truck)
    
#     return dolly_grazing



In [ ]:
# versions of above that have standard names for fields so I can underdstand

# def find_your_flow(dataframe, batch_size):
#     num_batches = (len(dataframe) // batch_size) + 1
#     all_texts = []

#     for i in range(0, num_batches, 20):
#         end_batch = min(i + 20, num_batches)
#         for batch_idx in tqdm(range(i, end_batch), desc=f"Processing batches {i+1}-{end_batch}"):
#             start_idx = batch_idx * batch_size
#             end_idx = min((batch_idx + 1) * batch_size, len(dataframe))
#             batch_df = dataframe.iloc[start_idx:end_idx]
            
#             batch_texts = [melodies(row) for _, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f"Processing batch {batch_idx+1}/{num_batches}")]
#             all_texts.extend(batch_texts)
    
#     return all_texts

# def dolly_tunes(texts, batch_size):
#     num_batches = (len(texts) // batch_size) + 1
#     tokenized_batches = []

#     for i in range(0, num_batches, 20):
#         end_batch = min(i + 20, num_batches)
#         for batch_idx in tqdm(range(i, end_batch), desc=f"Tokenizing batches {i+1}-{end_batch}"):
#             start_idx = batch_idx * batch_size
#             end_idx = min((batch_idx + 1) * batch_size, len(texts))
#             batch_texts = texts[start_idx:end_idx]
            
#             large_token = llama_3211b_tokenizer(batch_texts, return_tensors="tf", truncation=True, padding=True)
#             tokenized_batches.append(large_token)
    
#     return tokenized_batches


Dolly tunes that supports visualizations

In [9]:
import random
import logging
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Set up logging configuration
logging.basicConfig(filename='tokenization_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def dolly_tunes(texts, farm_trips):
    hay_barrels = (len(texts) // farm_trips) + 1
    dolly_grazing = []

    # Variables for analysis
    all_tokens = []
    token_lengths = []
    vocabulary_growth = []
    sample_indices = random.sample(range(len(texts)), min(100, len(texts)))

    for i in range(0, hay_barrels, 20):
        last_barrel = min(i + 20, hay_barrels)
        for barrel in tqdm(range(i, last_barrel), desc=f"Tokenizing batches {i+1}-{last_barrel}"):
            fresh_cut = barrel * farm_trips
            full_up = min((barrel + 1) * farm_trips, len(texts))
            full_barrel = texts[fresh_cut:full_up]
            
            # Tokenization of batch
            full_truck = llama_3211b_tokenizer(
                full_barrel,
                return_tensors="tf",
                truncation=True,
                padding=True
            )

            for idx, text in enumerate(full_barrel):
                tokens = llama_3211b_tokenizer.tokenize(text)
                token_ids = llama_3211b_tokenizer.convert_tokens_to_ids(tokens)

                # Collect tokens for analysis
                all_tokens.extend(tokens)
                token_lengths.extend([len(token) for token in tokens])
                
                # Log all tokens into file
                logging.info(f"Text: {text}\nTokens: {tokens}\nToken IDs: {token_ids}")

                # Print out matching rows of random sample using global index
                global_idx = fresh_cut + idx  
                if global_idx in sample_indices:
                    print("Tokens:", tokens)
                    print("Token IDs:", token_ids)

            dolly_grazing.append(full_truck)
            vocabulary_growth.append(len(set(all_tokens)))

    # Summarize Token Statistics
    summarize_token_statistics(all_tokens, token_lengths)

    # Token Frequency Analysis
    token_frequency_analysis(all_tokens)

    # Visualize Aggregated Data
    visualize_vocabulary_growth(vocabulary_growth)
    visualize_token_length_distribution(token_lengths)
    visualize_token_frequency_heatmap(all_tokens)

    return dolly_grazing

# Function to summarize token statistics
def summarize_token_statistics(all_tokens, token_lengths):
    avg_length = sum(token_lengths) / len(token_lengths) if len(token_lengths) > 0 else 0
    print(f"Total number of tokens: {len(all_tokens)}")
    print(f"Vocabulary size: {len(set(all_tokens))}")
    print(f"Average token length: {avg_length}")
    print(f"Minimum token length: {min(token_lengths)}")
    print(f"Maximum token length: {max(token_lengths)}")

# Function for Token Frequency Analysis
def token_frequency_analysis(all_tokens):
    token_counts = Counter(all_tokens)
    most_common_tokens = token_counts.most_common(20)
    print("\nMost Common Tokens:")
    for token, count in most_common_tokens:
        print(f"{token}: {count}")

# Function to visualize vocabulary growth
def visualize_vocabulary_growth(vocabulary_growth):
    plt.figure(figsize=(10, 6))
    plt.plot(vocabulary_growth)
    plt.xlabel('Batch Number')
    plt.ylabel('Vocabulary Size')
    plt.title('Vocabulary Growth Over Time')
    plt.show()

# Function to visualize token length distribution
def visualize_token_length_distribution(token_lengths):
    plt.figure(figsize=(10, 6))
    plt.hist(token_lengths, bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('Token Length')
    plt.ylabel('Frequency')
    plt.title('Token Length Distribution')
    plt.show()

# Function to visualize token frequency heatmap
def visualize_token_frequency_heatmap(all_tokens):
    token_counts = Counter(all_tokens)
    top_tokens = [token for token, _ in token_counts.most_common(20)]
    top_token_counts = [token_counts[token] for token in top_tokens]

    plt.figure(figsize=(10, 6))
    sns.heatmap([top_token_counts], annot=True, cmap='YlGnBu', xticklabels=top_tokens)
   


In [7]:
farm_trips = 50000
bp_text_values = find_your_flow(bp_text_values_df, farm_trips)

Processing batches 181-191: 100%|██████████| 11/11 [00:16<00:00,  1.46s/it]


In [10]:
bp_text_tokens = dolly_tunes(bp_text_values, farm_trips)

Tokenizing batches 1-20:  15%|█▌        | 3/20 [01:17<07:18, 25.81s/it]

Tokens: ['Track', 'ĠID', ':', 'Ġ', '178', '056', '49', ',', 'ĠTitle', ':', 'ĠBang', 's', 'ĠIn', 'ĠThe', 'ĠHead', ',', 'ĠArtist', ':', 'ĠVal', 'eri', 'Ã¸', 'ĠInn', 'Ã¸r', 'ta', ',', 'ĠArtist', 'ĠID', ':', 'Ġ', '647', '817', ',', 'ĠGenre', ':', 'ĠHard', 'ĠTechn', 'o', ',', 'ĠGenre', 'ĠID', ':', 'Ġ', '2', ',', 'ĠLabel', ':', 'ĠCar', 'bone', 'ĠRecords', ',', 'ĠLabel', 'ĠID', ':', 'Ġ', '591', '59', ',', 'ĠRelease', 'ĠDate', ':', 'Ġ', '202', '3', '-', '06', '-', '30', ',', 'ĠTrack', 'ĠURL', ':', 'Ġbeat', 'port', '.com', '/', 'track', '/b', 'angs', '-in', '-the', '-head', '/', '178', '056', '49', ',', 'ĠMix', ':', 'ĠOriginal', 'ĠMix', ',', 'ĠRemix', ':', 'ĠYes', ',', 'ĠRem', 'ixer', ':', 'ĠYes', ',', 'ĠDuration', ':', 'Ġ', '5', ':', '20', 'Ġminutes', ',', 'ĠBPM', ':', 'Ġ', '80', '.', '0', ',', 'ĠKey', 'ĠID', ':', 'Ġ', '3', '.', '0', ',', 'ĠMode', ':', 'Ġ', '0', '.', '0', ',', 'ĠVal', 'ence', ':', 'Ġ', '0', '.', '527', ',', 'ĠDance', 'ability', ':', 'Ġ', '0', '.', '686', ',', 'ĠEnergy', ':', '

Tokenizing batches 1-20:  25%|██▌       | 5/20 [02:11<06:37, 26.47s/it]

Tokens: ['Track', 'ĠID', ':', 'Ġ', '153', '921', '98', ',', 'ĠTitle', ':', 'ĠMy', 'ĠLittle', 'ĠFantasy', ',', 'ĠArtist', ':', 'ĠMad', 'agas', 'ca', ',', 'ĠArtist', 'ĠID', ':', 'Ġ', '744', '949', ',', 'ĠGenre', ':', 'ĠHard', 'ĠDance', 'Ġ/', 'ĠHardcore', ',', 'ĠGenre', 'ĠID', ':', 'Ġ', '8', ',', 'ĠLabel', ':', 'ĠChe', 'ek', 'y', 'ĠTracks', ',', 'ĠLabel', 'ĠID', ':', 'Ġ', '182', '33', ',', 'ĠRelease', 'ĠDate', ':', 'Ġ', '202', '1', '-', '07', '-', '16', ',', 'ĠTrack', 'ĠURL', ':', 'Ġbeat', 'port', '.com', '/', 'track', '/my', '-l', 'ittle', '-f', 'antasy', '/', '153', '921', '98', ',', 'ĠMix', ':', 'ĠRemix', ',', 'ĠRemix', ':', 'ĠYes', ',', 'ĠRem', 'ixer', ':', 'ĠYes', ',', 'ĠDuration', ':', 'Ġ', '5', ':', '39', 'Ġminutes', ',', 'ĠBPM', ':', 'Ġ', '150', '.', '0', ',', 'ĠKey', 'ĠID', ':', 'Ġ', '19', '.', '0', ',', 'ĠMode', ':', 'Ġ', '1', '.', '0', ',', 'ĠVal', 'ence', ':', 'Ġ', '0', '.', '522', ',', 'ĠDance', 'ability', ':', 'Ġ', '0', '.', '602', ',', 'ĠEnergy', ':', 'Ġ', '0', '.', '991', 

Tokenizing batches 1-20:  30%|███       | 6/20 [02:38<06:12, 26.64s/it]

...

Tokenizing batches 101-120:   0%|          | 0/20 [00:00<?, ?it/s]

Tokens: ['Track', 'ĠID', ':', 'Ġ', '164', '509', '5', ',', 'ĠTitle', ':', 'ĠDual', 'ĠBand', ',', 'ĠArtist', ':', 'ĠRico', 'ĠBon', 'etti', ',', 'ĠArtist', 'ĠID', ':', 'Ġ', '947', '72', ',', 'ĠGenre', ':', 'ĠHouse', ',', 'ĠGenre', 'ĠID', ':', 'Ġ', '5', ',', 'ĠLabel', ':', 'ĠP', 'ino', 'ĠMusic', ',', 'ĠLabel', 'ĠID', ':', 'Ġ', '857', '5', ',', 'ĠRelease', 'ĠDate', ':', 'Ġ', '201', '1', '-', '01', '-', '14', ',', 'ĠTrack', 'ĠURL', ':', 'Ġbeat', 'port', '.com', '/', 'track', '/d', 'ual', '-band', '/', '164', '509', '5', ',', 'ĠMix', ':', 'ĠOriginal', 'ĠMix', ',', 'ĠRemix', ':', 'ĠYes', ',', 'ĠRem', 'ixer', ':', 'ĠYes', ',', 'ĠDuration', ':', 'Ġ', '11', ':', '38', 'Ġminutes', ',', 'ĠBPM', ':', 'Ġ', '151', '.', '0', ',', 'ĠKey', 'ĠID', ':', 'Ġ', '6', '.', '0', ',', 'ĠMode', ':', 'Ġ', '0', '.', '0', ',', 'ĠVal', 'ence', ':', 'Ġ', '0', '.', '391', ',', 'ĠDance', 'ability', ':', 'Ġ', '0', '.', '806', ',', 'ĠEnergy', ':', 'Ġ', '0', '.', '899', ',', 'ĠSpeech', 'iness', ':', 'Ġ', '0', '.', '060', '

Tokenizing batches 101-120:  10%|█         | 2/20 [04:00<36:06, 120.35s/it]

Tokens: ['Track', 'ĠID', ':', 'Ġ', '178', '783', '09', ',', 'ĠTitle', ':', 'ĠBack', ',', 'ĠArtist', ':', 'ĠW', 'add', ',', 'ĠArtist', 'ĠID', ':', 'Ġ', '624', '782', ',', 'ĠGenre', ':', 'ĠOrganic', 'ĠHouse', 'Ġ/', 'ĠD', 'ownt', 'empo', ',', 'ĠGenre', 'ĠID', ':', 'Ġ', '93', ',', 'ĠLabel', ':', 'ĠTibet', 'ania', 'ĠRecords', ',', 'ĠLabel', 'ĠID', ':', 'Ġ', '914', '79', ',', 'ĠRelease', 'ĠDate', ':', 'Ġ', '202', '3', '-', '07', '-', '31', ',', 'ĠTrack', 'ĠURL', ':', 'Ġbeat', 'port', '.com', '/', 'track', '/back', '/', '178', '783', '09', ',', 'ĠMix', ':', 'ĠRemix', ',', 'ĠRemix', ':', 'ĠYes', ',', 'ĠRem', 'ixer', ':', 'ĠYes', ',', 'ĠDuration', ':', 'Ġ', '8', ':', '24', 'Ġminutes', ',', 'ĠBPM', ':', 'Ġ', '105', '.', '0', ',', 'ĠKey', 'ĠID', ':', 'Ġ', '9', '.', '0', ',', 'ĠMode', ':', 'Ġ', '0', '.', '0', ',', 'ĠVal', 'ence', ':', 'Ġ', '0', '.', '059', '4', ',', 'ĠDance', 'ability', ':', 'Ġ', '0', '.', '756', ',', 'ĠEnergy', ':', 'Ġ', '0', '.', '487', ',', 'ĠSpeech', 'iness', ':', 'Ġ', '0', '.

Tokenizing batches 101-120:  40%|████      | 8/20 [14:28<20:59, 105.00s/it]

### Cache tokenized data

In [ ]:
def pickled_cache(obj, filename_prefix, pickles):
    os.makedirs('cache_parts', exist_ok=True)
    for i in range(0, len(obj), pickles):
        with open(f'cache_parts/{filename_prefix}_part_{i // pickles}.pkl', 'wb') as part_file:
            pickle.dump(obj[i:i + pickles], part_file)

def pickle_jar(filename_prefix):
    parts = []
    i = 0
    while os.path.exists(f'cache_parts/{filename_prefix}_part_{i}.pkl'):
        with open(f'cache_parts/{filename_prefix}_part_{i}.pkl', 'rb') as part_file:
            parts.extend(pickle.load(part_file))
        i += 1
    return parts

In [ ]:
pickle_count = 50000
pickled_cache(bp_text_tokens, 'bp_text_tokens_cache')

In [ ]:

bp_text_tokens_cached = pickle_jar('bp_text_tokens_cache')

print("Tokenized output: ", bp_text_tokens_cached)

### Token Visualizations

#### Summarize Token Statistics

In [ ]:
#Token Length Statistics
from statistics import mean, median

all_token_lengths = [len(token) for tokens_batch in dolly_grazing for token in tokens_batch]

print("Mean Token Length:", mean(all_token_lengths))
print("Median Token Length:", median(all_token_lengths))
print("Minimum Token Length:", min(all_token_lengths))
print("Maximum Token Length:", max(all_token_lengths))

In [ ]:
#Vocabulary Size
unique_tokens = set([token for tokens_batch in dolly_grazing for token in tokens_batch])
print("Vocabulary Size:", len(unique_tokens))


#### Token Frequency Analysis

In [ ]:
from collections import Counter

# Count token frequencies
all_tokens = [token for tokens_batch in dolly_grazing for token in tokens_batch]
token_counts = Counter(all_tokens)

# Print the top 20 most common tokens
most_common_tokens = token_counts.most_common(20)
for token, count in most_common_tokens:
    print(f"Token: {token}, Frequency: {count}")

#### Visualize Aggregated Data

 Instead of focusing on individual tokens, visualize aggregated statistics. For instance:

Vocabulary Growth Curve: Plot the growth of unique tokens as you tokenize more data to understand how the vocabulary evolves with more input.
Token Length Distribution: Create a histogram or boxplot of token lengths.
Frequency Heatmaps: Use a heatmap to visualize how often specific tokens appear in different parts of the dataset

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

# Accumulate token frequencies over batches
token_frequency = defaultdict(int)

for batch in dolly_grazing:
    for tokens in batch['input_ids']:
        for token in tokens:
            token_frequency[token] += 1

# Get the top 20 most common tokens by frequency
sorted_tokens = sorted(token_frequency.items(), key=lambda x: x[1], reverse=True)[:20]

# Plot the token frequencies
tokens, frequencies = zip(*sorted_tokens)
plt.figure(figsize=(10, 6))
plt.bar(tokens, frequencies)
plt.xticks(rotation=45)
plt.xlabel('Token IDs')
plt.ylabel('Frequency')
plt.title('Top 20 Most Common Tokens')
plt.show()

### processing HTML into text tokens 

In [ ]:
def dolly_crawl(html_location):
    seeds = document_loaders.HTMLLoader(html_location)
    pellets = seeds.load()
    snack = pellets[0].page_content
    clean_cut = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunks = clean_cut.split_text(snack)
    tokenized_chunks = [llama_3211b_tokenizer(chunk, return_tensors="tf", truncation=True, padding=True) for chunk in chunks]

    return tokenized_chunks

radiance = "I need to add an html file here at some point"
tokens = dolly_crawl(radiance)
print(f"Tokenized output: {tokens}")
